In [1]:
from bs4 import BeautifulSoup
import urllib3
import re
import pandas as pd

In [2]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
http = urllib3.PoolManager()

In [26]:
mg_base_df = pd.read_csv(r"C:\Users\payoj.jain\Documents\projects\mg_base.csv", encoding='utf-8')

In [27]:
mg_base_df.head()

,Unnamed: 0,ACCOMMODATION_CODE,X,ACCOMMODATION_TYPE,ACCO_DESCRIPTION,PROVIDER_NAME,NUMBER_OF_PERSONS,BEDROOM_COUNT,BATHROOM_COUNT,HAS_WIFI,...,X2018.y,X2019.y,total2019,total2018,total2017,Total_HO,Total_BV,LATITUDE,LONGITUDE,max
0,1,AT-1050-01,1881,1,Apartment,Belvilla,4,1,1,1,...,432.358,0.0,0.0,432.358,3695.5980,3059.476,1068.4800,48.184128,16.367247,3695.5980
1,2,AT-1115-01,1950,3,Chalet,Belvilla,10,0,0,0,...,0.000,0.0,0.0,0.000,4474.4400,0.000,4474.4400,47.038075,10.610737,4474.4400
2,3,AT-1160-01,1996,1,Apartment,Belvilla,2,2,1,1,...,0.000,0.0,0.0,0.000,687.7720,329.472,358.3000,48.216322,16.312146,687.7720
3,4,AT-1160-02,1997,1,Apartment,Belvilla,2,1,1,1,...,0.000,0.0,0.0,0.000,642.2160,123.266,518.9500,48.216322,16.312146,642.2160
4,5,AT-1160-03,1998,1,Apartment,Belvilla,4,3,1,1,...,0.000,0.0,0.0,0.000,2632.7265,430.144,2202.5825,48.216320,16.312150,2632.7265


In [21]:
len(df.values)

22624

In [5]:
locations = df[['ACCOMMODATION_CODE','LATITUDE', 'LONGITUDE']]

In [6]:
splits = locations['ACCOMMODATION_CODE'].str.split("-")

In [7]:
locations['COUNTRY'] = pd.DataFrame(data=list(splits)[0:])[0]

C:\Users\payoj.jain\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
locations.head()

,ACCOMMODATION_CODE,LATITUDE,LONGITUDE,COUNTRY
0,AT-1050-01,48.184128,16.367247,AT
1,AT-1115-01,47.038075,10.610737,AT
2,AT-1160-01,48.216322,16.312146,AT
3,AT-1160-02,48.216322,16.312146,AT
4,AT-1160-03,48.216320,16.312150,AT


In [9]:
locations_with_elevation = pd.DataFrame(data=[], columns = {'ACCOMMODATION_CODE','LATITUDE', 'LONGITUDE', 'ELEVATION'})

In [10]:
country_names = locations['COUNTRY'].drop_duplicates()
countries = country_names.values
countries_list = list(countries)

In [11]:
len(countries_list)

24

In [12]:
remove_countries = ['IT', 'AT', 'BE', 'BQ', 'CH', 'CW', 'CZ', 'DE','DK' ,'ES']
for countr in remove_countries:
    countries_list.remove(countr)

In [13]:
index = 0
usernames = ['payoj','trojan','leo','myname', 'mike','galwayireland']
for country in countries_list:
    country_df = locations.query('COUNTRY == @country')
    country_lat_long_df = country_df[['ACCOMMODATION_CODE','LATITUDE', 'LONGITUDE']]
    print(country," has ",len(country_lat_long_df.values))
    elevation = []
    no_property = 0
    for acc, lat, long in country_lat_long_df.values:
        try:
            username = usernames[index]
            url = 'https://secure.geonames.org/srtm3JSON?lat='+str(lat)+'&lng='+str(long)+'&username='+username
            response = http.request('GET', url)
            soup = BeautifulSoup(response.data)
            elevation_arr = re.findall('"srtm3":(.+?),', str(soup))
            elevation.append([acc, lat, long, float(elevation_arr[0])])
            if no_property%500 == 0:
                print(no_property+1, " locations ", "scraped. Altitude of ", no_property+1,"st property ",elevation_arr[0])
            no_property += 1
        except(IndexError):
            index = (index+1)%6
            print(soup)
            username = usernames[index]
            print("Switched to user: ", username)
            continue
    
    df = pd.DataFrame(data=elevation, columns={'ACCOMMODATION_CODE','LATITUDE','LONGITUDE','ELEVATION'})
#     print(df.head())
#     print(len(df.values), len(country_df.values))
#     country_df = pd.merge(country_df, df, on='ACCOMMODATION_CODE',how='outer')
    locations_with_elevation = pd.concat([locations_with_elevation, df], axis = 0)

#     if(len(country_df.values) == len(elevation)):
#         country_df['ELEVATION'] = elevation
#         locations_with_elevation = pd.concat([locations_with_elevation, country_df], axis = 0)
#         print(locations_with_elevation.head())
#     else:
#         country_df = pd.merge(country_df, df, on='ACCOMMODATION_CODE',how='outer')

FR  has  3433
1  locations  scraped. Altitude of  1 st property  477
501  locations  scraped. Altitude of  501 st property  219
1001  locations  scraped. Altitude of  1001 st property  93
1501  locations  scraped. Altitude of  1501 st property  170
2001  locations  scraped. Altitude of  2001 st property  216
2501  locations  scraped. Altitude of  2501 st property  1750
3001  locations  scraped. Altitude of  3001 st property  309
GB  has  286
1  locations  scraped. Altitude of  1 st property  46
GR  has  297
1  locations  scraped. Altitude of  1 st property  145
HR  has  2367
1  locations  scraped. Altitude of  1 st property  166
501  locations  scraped. Altitude of  501 st property  21
<html><body><p>{"status":{"message":"the hourly limit of 1000 credits for payoj has been exceeded. Please throttle your requests or use the commercial service.","value":19}}</p></body></html>
Switched to user:  trojan
1001  locations  scraped. Altitude of  1001 st property  116
1501  locations  scraped. 

In [22]:
locations_with_elevation.to_csv(r"C:\Users\payoj.jain\Documents\projects\location_with_elevation3.csv", sep=",")

In [16]:
locations_with_elevation.head()

,LONGITUDE,LATITUDE,ACCOMMODATION_CODE,ELEVATION
0,FR-00000-02,45.559292,1.634630,477.0
1,FR-00001-04,47.166115,3.871967,284.0
2,FR-00001-06,44.181747,6.630488,1269.0
3,FR-00001-07,49.181744,-1.578830,8.0
4,FR-00001-09,49.067738,-1.478137,109.0


In [17]:
len(locations_with_elevation.values)

9326

In [24]:
acco_elevation_df = pd.read_csv(r"C:\Users\payoj.jain\Documents\projects\italy_elevation.csv")

In [25]:
acco_elevation_df.head()

,ACCOMMODATION_CODE,LATITUDE,LONGITUDE,ELEVATION
0,IT-98060-09,38.171273,14.887381,-32768
1,IT-45012-06,44.993605,12.162525,-2
2,IT-45012-08,44.993605,12.162525,-2
3,IT-47814-03,44.134182,12.482958,-1
4,IT-48124-01,44.384822,12.315475,-1


In [38]:
acco_elevation_df = acco_elevation_df.drop(['LATITUDE','LONGITUDE'], axis=1)

In [39]:
mg_base_df.head()

,Unnamed: 0,ACCOMMODATION_CODE,X,ACCOMMODATION_TYPE,ACCO_DESCRIPTION,PROVIDER_NAME,NUMBER_OF_PERSONS,BEDROOM_COUNT,BATHROOM_COUNT,HAS_WIFI,...,X2018.y,X2019.y,total2019,total2018,total2017,Total_HO,Total_BV,LATITUDE,LONGITUDE,max
0,1,AT-1050-01,1881,1,Apartment,Belvilla,4,1,1,1,...,432.358,0.0,0.0,432.358,3695.5980,3059.476,1068.4800,48.184128,16.367247,3695.5980
1,2,AT-1115-01,1950,3,Chalet,Belvilla,10,0,0,0,...,0.000,0.0,0.0,0.000,4474.4400,0.000,4474.4400,47.038075,10.610737,4474.4400
2,3,AT-1160-01,1996,1,Apartment,Belvilla,2,2,1,1,...,0.000,0.0,0.0,0.000,687.7720,329.472,358.3000,48.216322,16.312146,687.7720
3,4,AT-1160-02,1997,1,Apartment,Belvilla,2,1,1,1,...,0.000,0.0,0.0,0.000,642.2160,123.266,518.9500,48.216322,16.312146,642.2160
4,5,AT-1160-03,1998,1,Apartment,Belvilla,4,3,1,1,...,0.000,0.0,0.0,0.000,2632.7265,430.144,2202.5825,48.216320,16.312150,2632.7265


In [40]:
mg_base_with_elevation = pd.merge(mg_base_df, acco_elevation_df, on='ACCOMMODATION_CODE', how="left")

In [47]:
mg_base_with_elevation.head()

,Unnamed: 0,ACCOMMODATION_CODE,X,ACCOMMODATION_TYPE,ACCO_DESCRIPTION,PROVIDER_NAME,NUMBER_OF_PERSONS,BEDROOM_COUNT,BATHROOM_COUNT,HAS_WIFI,...,X2019.y,total2019,total2018,total2017,Total_HO,Total_BV,LATITUDE,LONGITUDE,max,ELEVATION
0,1,AT-1050-01,1881,1,Apartment,Belvilla,4,1,1,1,...,0.0,0.0,432.358,3695.5980,3059.476,1068.4800,48.184128,16.367247,3695.5980,207.0
1,2,AT-1115-01,1950,3,Chalet,Belvilla,10,0,0,0,...,0.0,0.0,0.000,4474.4400,0.000,4474.4400,47.038075,10.610737,4474.4400,1444.0
2,3,AT-1160-01,1996,1,Apartment,Belvilla,2,2,1,1,...,0.0,0.0,0.000,687.7720,329.472,358.3000,48.216322,16.312146,687.7720,232.0
3,4,AT-1160-02,1997,1,Apartment,Belvilla,2,1,1,1,...,0.0,0.0,0.000,642.2160,123.266,518.9500,48.216322,16.312146,642.2160,232.0
4,5,AT-1160-03,1998,1,Apartment,Belvilla,4,3,1,1,...,0.0,0.0,0.000,2632.7265,430.144,2202.5825,48.216320,16.312150,2632.7265,232.0


In [51]:
mg_base_with_elevation.to_csv(r"C:\Users\payoj.jain\Documents\projects\mg_base_with_elevation.csv")